# Loading text pre-processing model
* https://github.com/UKPLab/sentence-transformers
* https://arxiv.org/abs/1908.10084

In [ ]:
!pip install git+https://github.com/rmarcacini/sentence-transformers
!pip install gdown
!gdown https://drive.google.com/uc?id=1NV5t1YhyyOzMF5zAovfbSLdZZLvqrfZ_
!unzip distiluse-base-multilingual-cased.zip -d language_model
from sentence_transformers import SentenceTransformer, LoggingHandler
import numpy as np
import logging

np.set_printoptions(threshold=100)
logging.basicConfig(format='%(asctime)s - %(message)s',datefmt='%Y-%m-%d %H:%M:%S',level=logging.INFO,handlers=[LoggingHandler()])

language_model = SentenceTransformer('./language_model')

  Cloning https://github.com/rmarcacini/sentence-transformers to /tmp/pip-req-build-pe30i3uy
  Running command git clone -q https://github.com/rmarcacini/sentence-transformers /tmp/pip-req-build-pe30i3uy
     |████████████████████████████████| 890kB 7.1MB/s 
     |████████████████████████████████| 1.2MB 33.4MB/s 
     |████████████████████████████████| 901kB 49.6MB/s 
     |████████████████████████████████| 3.0MB 50.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.6-cp37-none-any.whl size=101897 sha256=31d4a347015b41e8e471988697fced179b7f81e731265580cda4d9e7917f3563
  Stored in directory: /tmp/pip-ephem-wheel-cache-ne5vrfpn/wheels/88/3c/66/55ee9fb698480d5a5116a8257c15dc363323e4922fb8ad361b
Successfully built sentence-transformers
Downloading...
From: https://drive.google.com/uc?id=1NV5t1YhyyOzMF5zAovfbSLdZZLvqrfZ_
To: /content/distiluse-base-multilingual-cased.zip
504MB [00:04, 120MB/s]
Archive:  distiluse-base-multilingual-cased.zip
   creating: lan

# Loading requirements candidates

In [ ]:
!gdown --id 1O-R3Zz9li8SYDcFumPsDN9txA3L2QixG

Downloading...
From: https://drive.google.com/uc?id=1O-R3Zz9li8SYDcFumPsDN9txA3L2QixG
To: /content/requirement.csv
2.57MB [00:00, 81.7MB/s]


In [ ]:
import pandas as pd

df_data = pd.read_csv('requirement.csv')
df_data

,requirement_text
0,bought
1,support group orders with individual payments
2,cook more
3,process credit cards payments
4,cash
...,...
149630,intolerance
149631,Pooorer
149632,contraversial
149633,Worst.spoiledfood


Generate a feature vector for each requirement candidate

In [ ]:
df_data['embedding'] = list(language_model.encode(df_data['requirement_text'].to_list()))

In [ ]:
df_data

,requirement_text,embedding
0,bought,"[0.037175164, 0.02746418, -0.027728138, 0.0300..."
1,support group orders with individual payments,"[-0.06067936, -0.04864118, -0.0040709833, -0.0..."
2,cook more,"[0.011025423, -0.014061921, 0.009416151, -0.01..."
3,process credit cards payments,"[-0.068453215, 0.0036510304, 0.0064596976, 0.0..."
4,cash,"[0.044530906, -0.011020444, -0.049469605, -0.0..."
...,...,...
149630,intolerance,"[-0.018483961, -0.049156286, -0.00084916595, -..."
149631,Pooorer,"[0.039778505, -0.035537004, -0.036012214, -0.0..."
149632,contraversial,"[-0.025207547, -0.04103427, -0.056060143, -0.0..."
149633,Worst.spoiledfood,"[-0.0016031228, 0.010851361, 0.05258863, 0.016..."


# Grouping candidates based on feature vector

* https://dl.acm.org/doi/book/10.5555/3235282 (capitulo de livro)
* https://www.researchgate.net/publication/268289418_A_Survey_of_Text_Clustering_Algorithms/link/55c5667a08aea2d9bdc39b6b/download 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

from sklearn.metrics import silhouette_samples
import numpy as np

X = np.array(df_data['embedding'].to_list())
kmeans = MiniBatchKMeans(n_clusters=300, random_state=0, init='random',verbose=1,batch_size=1000,max_no_improvement=1000).fit(X)
df_data['cluster'] = kmeans.labels_

Init 1/3 with method: random
Inertia for init 1/3: 1126.972290
Init 2/3 with method: random
Inertia for init 2/3: 1110.588623
Init 3/3 with method: random
Inertia for init 3/3: 1122.899902
Minibatch iteration 1/15000: mean batch inertia: 0.432621, ewa inertia: 0.432621 
Minibatch iteration 2/15000: mean batch inertia: 0.418971, ewa inertia: 0.432439 
Minibatch iteration 3/15000: mean batch inertia: 0.404777, ewa inertia: 0.432069 
Minibatch iteration 4/15000: mean batch inertia: 0.400175, ewa inertia: 0.431643 
Minibatch iteration 5/15000: mean batch inertia: 0.399089, ewa inertia: 0.431208 
Minibatch iteration 6/15000: mean batch inertia: 0.402014, ewa inertia: 0.430817 
Minibatch iteration 7/15000: mean batch inertia: 0.388100, ewa inertia: 0.430246 
Minibatch iteration 8/15000: mean batch inertia: 0.391809, ewa inertia: 0.429733 
Minibatch iteration 9/15000: mean batch inertia: 0.394927, ewa inertia: 0.429267 
[MiniBatchKMeans] Reassigning 32 cluster centers.
Minibatch iteration 10/

In [ ]:
df_data

,requirement_text,embedding,cluster
0,bought,"[0.037175164, 0.02746418, -0.027728138, 0.0300...",60
1,support group orders with individual payments,"[-0.06067936, -0.04864118, -0.0040709833, -0.0...",2
2,cook more,"[0.011025423, -0.014061921, 0.009416151, -0.01...",31
3,process credit cards payments,"[-0.068453215, 0.0036510304, 0.0064596976, 0.0...",47
4,cash,"[0.044530906, -0.011020444, -0.049469605, -0.0...",178
...,...,...,...
149630,intolerance,"[-0.018483961, -0.049156286, -0.00084916595, -...",89
149631,Pooorer,"[0.039778505, -0.035537004, -0.036012214, -0.0...",67
149632,contraversial,"[-0.025207547, -0.04103427, -0.056060143, -0.0...",77
149633,Worst.spoiledfood,"[-0.0016031228, 0.010851361, 0.05258863, 0.016...",220


In [ ]:
df_data[df_data.cluster==47]

2021-06-18 17:36:34 - NumExpr defaulting to 2 threads.


,requirement_text,embedding,cluster
3,process credit cards payments,"[-0.068453215, 0.0036510304, 0.0064596976, 0.0...",47
118,credit card processing,"[-0.05089237, 0.009707626, -0.010439861, 0.026...",47
181,card,"[-0.03323663, -0.0034145599, -0.03726258, 0.00...",47
188,debit card,"[-0.026913177, -0.02715102, -0.023486247, 0.02...",47
233,credit card,"[-0.041877277, -0.00074380304, -0.024592008, 0...",47
...,...,...,...
148122,hack your card,"[-0.026397306, 0.006570547, -0.032916613, 0.02...",47
148680,show card option,"[0.005101489, 0.038082905, 0.0007375727, 0.074...",47
148681,shows card options,"[0.011716591, 0.027970335, 0.0015033563, 0.080...",47
148926,payments through food cards,"[-0.09067322, -0.02396723, -0.0015926573, -0.0...",47


In [ ]:
df_data[df_data.cluster==0]

,requirement_text,embedding,cluster
683,update my contact info,"[-0.08187661, 0.042463683, 0.0079845, 0.074037...",0
735,update the details,"[-0.09368116, 0.04675122, -0.024632866, 0.0907...",0
953,update my profile,"[-0.06370778, 0.053309686, -0.02890365, 0.0967...",0
1046,update contact info,"[-0.0848206, 0.042936467, 0.013918944, 0.08375...",0
1073,update contact details,"[-0.10833922, 0.05706967, 0.0029367208, 0.0754...",0
...,...,...,...
143324,update its information,"[-0.054805808, 0.0077626314, -0.0113326665, 0....",0
145100,updated on restaurant information,"[-0.09538414, 0.0037518688, 0.022917407, 0.064...",0
145348,update correct contact details,"[-0.10763693, 0.03326906, 0.013415819, 0.03858...",0
146139,updating all my bookmarks,"[0.018984891, 0.038653534, -0.059202563, 0.030...",0


In [ ]:
df_data[df_data.cluster==220]

,requirement_text,embedding,cluster
1120,saying rude words,"[0.008787694, -0.011050601, -0.008847884, 0.02...",220
3447,anger boosting,"[-0.015631177, 0.018261738, -0.043654673, 0.00...",220
3806,SickMessagingFeature,"[-0.030806001, 0.015994329, 0.018026613, 0.073...",220
3832,Commit suicide,"[0.0023929467, -0.03302312, -0.022682564, -0.0...",220
3958,translate malay to English,"[-0.030080566, -0.051196765, 0.015247984, 0.04...",220
...,...,...,...
149445,Disturbance Aap,"[0.030520817, 0.015248947, -0.021817314, 0.037...",220
149471,Animal abuse,"[0.0142560145, -0.018439716, 0.092928335, -0.0...",220
149472,BoycottChina,"[-0.0003806208, -0.012572662, -0.005591418, 0....",220
149633,Worst.spoiledfood,"[-0.0016031344, 0.010851323, 0.052588638, 0.01...",220


## Identifying well-placed candidates in your cluster

* https://en.wikipedia.org/wiki/Silhouette_(clustering)



In [ ]:
from sklearn.metrics import silhouette_samples
import numpy as np

df_data['silhouette'] = silhouette_samples(X,  kmeans.labels_)

In [ ]:
df_data

,requirement_text,embedding,cluster,silhouette
0,bought,"[0.037175164, 0.02746418, -0.027728138, 0.0300...",60,0.168153
1,support group orders with individual payments,"[-0.06067936, -0.04864118, -0.0040709833, -0.0...",2,-0.000717
2,cook more,"[0.011025423, -0.014061921, 0.009416151, -0.01...",31,0.115923
3,process credit cards payments,"[-0.068453215, 0.0036510304, 0.0064596976, 0.0...",47,0.072541
4,cash,"[0.044530906, -0.011020444, -0.049469605, -0.0...",178,0.103650
...,...,...,...,...
149630,intolerance,"[-0.018483961, -0.049156286, -0.00084916595, -...",89,-0.089331
149631,Pooorer,"[0.039778505, -0.035537004, -0.036012214, -0.0...",67,0.070369
149632,contraversial,"[-0.025207547, -0.04103427, -0.056060143, -0.0...",77,-0.016629
149633,Worst.spoiledfood,"[-0.0016031228, 0.010851361, 0.05258863, 0.016...",220,-0.119437


In [ ]:
# ordenando pelo melhor silhouette (o topo eh um candidato para nomear o cluster)
df_data[df_data.cluster==0].sort_values(by='silhouette', ascending=False)

,requirement_text,embedding,cluster,silhouette
21021,me update my profile info,"[-0.059015255, 0.039536573, -0.024006462, 0.09...",0,0.139901
51741,update my account information,"[-0.071642414, 0.029500071, 0.015441682, 0.076...",0,0.135037
109585,telling me to update my billing information,"[-0.05916071, 0.027630934, 0.0034455154, 0.067...",0,0.133773
51221,update me billing info,"[-0.06451203, 9.2335395e-06, -0.004199724, 0.0...",0,0.133121
57937,update my billing settings,"[-0.07343577, 0.032138404, 0.011183555, 0.0634...",0,0.132948
...,...,...,...,...
56696,update my own credit card details,"[-0.110442355, 0.02671429, -0.016706113, 0.056...",0,-0.060873
1471,update contact number,"[-0.09658992, 0.030962607, -0.036351435, 0.049...",0,-0.061498
52990,update my credit card details,"[-0.09660519, 0.031507492, -0.022618154, 0.068...",0,-0.067565
117503,update phone number in profile,"[-0.104245074, 0.03133098, -0.054412168, 0.056...",0,-0.068673


In [ ]:
# ordenando pelo melhor silhouette (o topo eh um candidato para nomear o cluster)
df_data[df_data.cluster==47].sort_values(by='silhouette', ascending=False)

,requirement_text,embedding,cluster,silhouette
66260,Payment cards,"[-0.039152864, -0.03024029, -0.052215483, 0.04...",47,0.145860
1912,credit cards,"[-0.027322367, -0.011047636, -0.035997145, 0.0...",47,0.144391
106801,Credit Cards,"[-0.02143452, 0.0074433074, -0.037838705, 0.04...",47,0.142757
23059,Credit cards,"[-0.019102467, 0.002080985, -0.04395124, 0.031...",47,0.142380
39648,creditcards,"[-0.024997879, -0.006144399, -0.04340581, 0.00...",47,0.142368
...,...,...,...,...
72752,deduct from debit card,"[-0.049438298, -0.04727059, -0.022030044, 0.03...",47,-0.057574
33574,Order through card,"[-0.024310656, -0.0153507, 0.029571172, 0.0688...",47,-0.057681
42675,auto goes off your card,"[0.02292111, -0.032058626, -0.056788508, -0.00...",47,-0.062883
106809,clears card,"[-0.04509657, 0.010905642, -0.015270534, 0.063...",47,-0.075889


In [ ]:
df_data[['requirement_text','cluster','silhouette']].to_csv('requirements_clusters.csv')

## Extracting embeddings

In [ ]:
!gdown --id 1uuKUhOp68tH5yeUq5J4rjcIvjGq6PgCR

Downloading...
From: https://drive.google.com/uc?id=1uuKUhOp68tH5yeUq5J4rjcIvjGq6PgCR
To: /content/requirements_clusters-2.csv
5.79MB [00:00, 26.4MB/s]


In [ ]:
import pandas as pd

df_data = pd.read_csv('requirements_clusters-2.csv')
df_data

,Unnamed: 0,requirement_text,cluster,silhouette
0,0,bought,60,0.168153
1,1,support group orders with individual payments,2,-0.000717
2,2,cook more,31,0.115923
3,3,process credit cards payments,47,0.072541
4,4,cash,178,0.103650
...,...,...,...,...
149630,149630,intolerance,89,-0.089331
149631,149631,Pooorer,67,0.070369
149632,149632,contraversial,77,-0.016629
149633,149633,Worst.spoiledfood,220,-0.119437


In [ ]:
df_data['embedding'] = list(language_model.encode(df_data['requirement_text'].to_list()))
df_data

,Unnamed: 0,requirement_text,cluster,silhouette,embedding
0,0,bought,60,0.168153,"[0.037175164, 0.02746418, -0.027728138, 0.0300..."
1,1,support group orders with individual payments,2,-0.000717,"[-0.06067936, -0.04864118, -0.0040709833, -0.0..."
2,2,cook more,31,0.115923,"[0.011025423, -0.014061921, 0.009416151, -0.01..."
3,3,process credit cards payments,47,0.072541,"[-0.068453215, 0.0036510304, 0.0064596976, 0.0..."
4,4,cash,178,0.103650,"[0.044530906, -0.011020444, -0.049469605, -0.0..."
...,...,...,...,...,...
149630,149630,intolerance,89,-0.089331,"[-0.018483961, -0.049156286, -0.00084916595, -..."
149631,149631,Pooorer,67,0.070369,"[0.039778505, -0.035537004, -0.036012214, -0.0..."
149632,149632,contraversial,77,-0.016629,"[-0.025207547, -0.04103427, -0.056060143, -0.0..."
149633,149633,Worst.spoiledfood,220,-0.119437,"[-0.0016031228, 0.010851361, 0.05258863, 0.016..."


In [ ]:
df_emb = pd.DataFrame(np.array(df_data.embedding.to_list()))
df_emb

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0.037175,0.027464,-0.027728,0.030051,-0.047393,-0.070054,-0.048226,-0.001437,-0.070056,0.002270,-0.001362,0.036649,-0.072419,-0.007883,0.068679,-0.002946,0.012186,0.038906,-0.021418,-0.023533,-0.009805,0.013207,0.007916,0.020905,0.006889,-0.027638,0.100307,-0.021532,-0.022688,-0.013657,-0.038405,0.022515,0.036805,-0.072783,0.041203,-0.096467,-0.034794,-0.027598,-0.009310,-0.018149,...,-0.069107,-0.002569,0.032430,0.048271,0.018485,0.032876,0.019514,0.040756,-0.003705,-0.008199,0.086558,0.004860,-0.072653,0.001134,-0.056511,0.021771,0.050127,-0.023038,-0.031581,0.003559,0.075101,0.016607,-0.007554,-0.005609,-0.006435,-0.028097,0.089207,-0.042285,-0.007975,-0.018953,0.114981,0.002194,0.026171,-0.014171,-0.065693,-0.061732,0.005795,-0.047110,0.029597,-0.036436
1,-0.060679,-0.048641,-0.004071,-0.040379,0.098232,-0.038001,-0.094894,0.035263,-0.023258,0.002307,0.002671,0.054337,0.024883,-0.025773,0.040958,0.043604,-0.010058,0.090123,-0.021185,-0.021610,0.044164,0.025283,0.002482,-0.031927,-0.065900,0.044052,0.003338,0.035657,-0.017699,-0.024205,0.025365,-0.045748,-0.013884,0.055764,0.048973,0.010396,0.011026,0.040924,-0.085098,-0.062968,...,-0.000794,-0.066317,0.060900,0.039245,-0.076814,-0.049168,-0.084025,0.002961,0.055662,-0.020807,-0.022711,-0.072616,-0.013577,-0.057921,0.015732,0.061590,-0.009314,0.058921,0.048101,0.009812,0.004249,-0.005354,0.037686,-0.091458,-0.090559,-0.039749,-0.022394,-0.017809,0.008167,-0.031706,0.002936,0.030972,-0.063914,-0.004490,0.022693,-0.013712,-0.009632,-0.001394,0.015019,0.023095
2,0.011025,-0.014062,0.009416,-0.019354,0.022201,0.047040,-0.028377,-0.052779,0.035758,-0.006486,-0.012642,0.019980,0.085202,-0.026478,0.064787,-0.022867,-0.093834,0.007117,-0.000794,-0.044362,-0.054917,-0.082330,-0.126892,0.061318,0.065384,-0.022290,-0.008654,-0.050452,0.027036,0.037346,-0.028986,0.056518,0.025427,-0.096483,0.023784,-0.051210,0.033010,0.012809,-0.004020,-0.004697,...,-0.019233,0.069105,0.039556,0.096254,-0.078774,0.039580,-0.001324,0.013485,-0.030752,-0.027498,0.041125,-0.028859,0.004331,0.020720,0.024748,-0.026894,0.062968,0.041143,0.013779,-0.000033,0.081710,0.007116,0.043038,0.031978,-0.005041,0.037950,0.032439,-0.071458,0.010836,0.040630,0.028821,-0.059694,0.008614,0.039775,0.050134,-0.067006,-0.151899,-0.036825,0.057874,0.057099
3,-0.068453,0.003651,0.006460,0.017188,0.028796,-0.048799,-0.051305,-0.001819,0.000649,0.033372,0.020469,0.067580,0.103719,-0.006749,-0.014027,0.060040,0.024211,-0.004119,-0.042480,-0.014482,0.063062,0.005483,-0.048422,-0.026761,-0.007421,0.032666,0.084156,0.002658,-0.056527,-0.053676,0.004661,-0.024201,0.048757,-0.040491,0.020864,0.037376,0.031301,0.003328,-0.079259,-0.014873,...,-0.054736,-0.070433,0.001755,-0.045277,-0.058274,0.015089,-0.016516,0.003814,-0.036066,-0.002048,0.029648,-0.073664,0.004622,0.019941,-0.067286,0.037086,-0.006820,-0.041382,-0.021345,0.053043,0.054233,0.038433,-0.017690,-0.039641,-0.012975,0.009827,0.038023,-0.035459,0.030627,-0.049873,0.033656,-0.027018,-0.003397,0.003600,0.028135,-0.012573,-0.053834,0.036301,0.007382,0.016326
4,0.044531,-0.011020,-0.049470,-0.040506,-0.046790,0.021136,0.013163,-0.032344,0.004080,-0.026158,-0.010939,0.022662,-0.002003,0.035570,0.061825,-0.019050,-0.022739,-0.020602,-0.032855,-0.037949,0.058830,0.046838,0.041907,-0.027975,0.029107,0.000487,0.056961,0.019094,0.038082,-0.037195,-0.038054,0.035368,-0.010014,-0.081853,0.019739,-0.097666,0.000508,0.000760,-0.006496,-0.011442,...,-0.036261,0.026595,-0.054722,0.015490,-0.062024,-0.027291,-0.009883,0.019979,0.001749,-0.040412,0.059668,-0.025110,0.033579,-0.038713,-0.076423,0.013667,-0.023482,0.007390,0.004022,0.007923,0.063995,0.026617,-0.002904,-0.022977,-0.018080,0.008585,0.044995,-0.062632,0.00184

In [ ]:
cols = df_emb.columns
df_emb[cols] = df_emb[cols].applymap(lambda x: '{0:.3f}'.format(x))

In [ ]:
df_emb

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0.037,0.027,-0.028,0.030,-0.047,-0.070,-0.048,-0.001,-0.070,0.002,-0.001,0.037,-0.072,-0.008,0.069,-0.003,0.012,0.039,-0.021,-0.024,-0.010,0.013,0.008,0.021,0.007,-0.028,0.100,-0.022,-0.023,-0.014,-0.038,0.023,0.037,-0.073,0.041,-0.096,-0.035,-0.028,-0.009,-0.018,...,-0.069,-0.003,0.032,0.048,0.018,0.033,0.020,0.041,-0.004,-0.008,0.087,0.005,-0.073,0.001,-0.057,0.022,0.050,-0.023,-0.032,0.004,0.075,0.017,-0.008,-0.006,-0.006,-0.028,0.089,-0.042,-0.008,-0.019,0.115,0.002,0.026,-0.014,-0.066,-0.062,0.006,-0.047,0.030,-0.036
1,-0.061,-0.049,-0.004,-0.040,0.098,-0.038,-0.095,0.035,-0.023,0.002,0.003,0.054,0.025,-0.026,0.041,0.044,-0.010,0.090,-0.021,-0.022,0.044,0.025,0.002,-0.032,-0.066,0.044,0.003,0.036,-0.018,-0.024,0.025,-0.046,-0.014,0.056,0.049,0.010,0.011,0.041,-0.085,-0.063,...,-0.001,-0.066,0.061,0.039,-0.077,-0.049,-0.084,0.003,0.056,-0.021,-0.023,-0.073,-0.014,-0.058,0.016,0.062,-0.009,0.059,0.048,0.010,0.004,-0.005,0.038,-0.091,-0.091,-0.040,-0.022,-0.018,0.008,-0.032,0.003,0.031,-0.064,-0.004,0.023,-0.014,-0.010,-0.001,0.015,0.023
2,0.011,-0.014,0.009,-0.019,0.022,0.047,-0.028,-0.053,0.036,-0.006,-0.013,0.020,0.085,-0.026,0.065,-0.023,-0.094,0.007,-0.001,-0.044,-0.055,-0.082,-0.127,0.061,0.065,-0.022,-0.009,-0.050,0.027,0.037,-0.029,0.057,0.025,-0.096,0.024,-0.051,0.033,0.013,-0.004,-0.005,...,-0.019,0.069,0.040,0.096,-0.079,0.040,-0.001,0.013,-0.031,-0.027,0.041,-0.029,0.004,0.021,0.025,-0.027,0.063,0.041,0.014,-0.000,0.082,0.007,0.043,0.032,-0.005,0.038,0.032,-0.071,0.011,0.041,0.029,-0.060,0.009,0.040,0.050,-0.067,-0.152,-0.037,0.058,0.057
3,-0.068,0.004,0.006,0.017,0.029,-0.049,-0.051,-0.002,0.001,0.033,0.020,0.068,0.104,-0.007,-0.014,0.060,0.024,-0.004,-0.042,-0.014,0.063,0.005,-0.048,-0.027,-0.007,0.033,0.084,0.003,-0.057,-0.054,0.005,-0.024,0.049,-0.040,0.021,0.037,0.031,0.003,-0.079,-0.015,...,-0.055,-0.070,0.002,-0.045,-0.058,0.015,-0.017,0.004,-0.036,-0.002,0.030,-0.074,0.005,0.020,-0.067,0.037,-0.007,-0.041,-0.021,0.053,0.054,0.038,-0.018,-0.040,-0.013,0.010,0.038,-0.035,0.031,-0.050,0.034,-0.027,-0.003,0.004,0.028,-0.013,-0.054,0.036,0.007,0.016
4,0.045,-0.011,-0.049,-0.041,-0.047,0.021,0.013,-0.032,0.004,-0.026,-0.011,0.023,-0.002,0.036,0.062,-0.019,-0.023,-0.021,-0.033,-0.038,0.059,0.047,0.042,-0.028,0.029,0.000,0.057,0.019,0.038,-0.037,-0.038,0.035,-0.010,-0.082,0.020,-0.098,0.001,0.001,-0.006,-0.011,...,-0.036,0.027,-0.055,0.015,-0.062,-0.027,-0.010,0.020,0.002,-0.040,0.060,-0.025,0.034,-0.039,-0.076,0.014,-0.023,0.007,0.004,0.008,0.064,0.027,-0.003,-0.023,-0.018,0.009,0.045,-0.063,0.002,-0.069,0.046,-0.024,0.042,0.002,-0.032,-0.054,0.018,0.014,0.056,-0.035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149630,-0.018,-0.049,-0.001,-0.055,-0.048,-0.014,0.023,-0.030,-0.050,-0.052,-0.010,-0.038,-0.028,0.029,0.075,-0.021,0.026,-0.013,0.039,0.012,0.001,0.069,-0.033,0.011,0.035,0.042,0.003,0.007,-0.026,0.034,-0.056,-0.016,0.106,-0.046,-0.050,-0.052,-0.086,0.012,-0.042,0.008,...,-0.033,0.038,-0.041,0.020,0.072,0.019,0.038,0.023,0.012,-0.035,0.059,-0.017,0.063,-0.011,-0.007,0.006,0.048,0.031,0.043,-0.016,0.101,-0.012,-0.001,0.031,0.112,-0.007,-0.004,-0.066,0.005,-0.028,0.025,-0.060,0.017,-0.054,-0.059,-0.096,-0.039,-0.029,0.006,-0.023
149631,0.040,-0.036,-0.036,-0.034,-0.048,0.025,-0.003,0.008,-0.003,-0.036,-0.069,0.042,-0.029,0.006,0.018,-0.031,0.017,-0.013,-0.051,-0.012,-0.013,0.033,-0.003,-0.021,0.008,0.041,-0.041,-0.022,-0.029,-0.004,0.009,0.046,-0.020,-0.048

In [ ]:
df_emb.head(8000).to_csv('embeddings.tsv',sep="\t",header=None,index=False)

In [ ]:
df_data[['embedding']].sample(8000).to_csv('embeddings.tsv',sep="\t",header=None,index=False)

In [ ]:
df_data[['cluster','requirement_text']].sample(8000).to_csv('metadata.tsv',sep="\t",index=False)